### Imports

In [1]:
import mediapipe as mp
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import cv2
import shutil
from glob import glob

### Functions

In [2]:
def mediapipe_inference(frame):
    results = pose.process(frame)
    frame_list1 = []
    point_coords_landmark_interest = []
    start_index = 0
    if results.pose_landmarks:
        for landmark in results.pose_landmarks.landmark:
            if start_index in indices_landmark_interest:
                if landmark.visibility>=0:
                    image_hight, image_width, _ = frame.shape
                    frame = cv2.circle(frame, (int(landmark.x*image_width), int(landmark.y*image_hight)), radius=5, color = (0,255,0), thickness=-1)   
                    frame_list1.append(landmark.x*image_width)
                    frame_list1.append(landmark.y*image_hight)
                    point_coords_landmark_interest.append((int(round(landmark.x*image_width)),int(round(landmark.y*image_hight))))
                    start_index = start_index+1
                else:
                    frame_list1.append(np.NaN)
                    frame_list1.append(np.NaN)
                    start_index = start_index+1
            else: 
                start_index = start_index+1
    else:
        frame_list1 = np.zeros(number_keypoints_to_detect*2)
        frame_list1[:] = np.nan    
        
    return(frame_list1,frame)

### Variables

In [3]:
actors = ['Adri', 'Xavi', 'Mireia']
indexes = ['01','02','03','04','05']


#Select the number of keypoints to detect 
number_keypoints_to_detect = 6

#Select the indices of the landmarks of interest to detect from the output of the previous cell
indices_landmark_interest = [11, 12, 13, 14, 15 ,16]

#Loading mediapipe pose detector 
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(model_complexity=0,min_detection_confidence=0.2,min_tracking_confidence=0.5)

### Code

In [ ]:
#CALDRIA AFEGIR EL TIMESTAMP DEL ARXIU
for actor in actors:
    for index in indexes:
            df_2D = pd.DataFrame(columns=np.arange(0,int(number_keypoints_to_detect*2)+1))
            frames_path = glob('../../data_export_1/Kinect/'+actor+'/'+actor+index+'/mp4_splitted/*.jpg')
            
            for frame_path in frames_path:
                frame = cv2.imread(frame_path)       
                timecode = float(frame_path.split('/')[-1].split('_')[-1].split('.jpg')[0])

                try: 
                    detected_keypoints, frame = mediapipe_inference(frame)
                    detected_keypoints.insert(0,timecode)
                    df_2D = df_2D.append(pd.DataFrame(np.array(detected_keypoints).reshape(1,-1)))
                    cv2.imwrite('../../data_export_1/Kinect/'+actor+'/'+actor+index+'/mediapipe_processed_frames/'+frame_path.split('\\')[-1],frame)
                except:
                    #Si peta per qualsevol motiu, afegeix una fila de nans
                    nan_array = np.empty((1,13))
                    nan_array[:] = np.nan
                    df_2D = df_2D.append(pd.DataFrame(nan_array))
                    print(frame_path)
            df_2D.to_csv('../../data_export_1/Kinect/'+actor+'/'+actor+index+'/mediapipe_processed_frames/mediapipe_output_'+actor+index+'.csv',header=None, index=None )

../../data_export_1/Kinect/Adri/Adri01/mp4_splitted\Frame_0668_14.379220000000142.jpg
../../data_export_1/Kinect/Adri/Adri01/mp4_splitted\Frame_0670_14.445900000000144.jpg
../../data_export_1/Kinect/Adri/Adri01/mp4_splitted\Frame_0671_14.479240000000145.jpg
../../data_export_1/Kinect/Adri/Adri01/mp4_splitted\Frame_0673_14.545920000000146.jpg
../../data_export_1/Kinect/Adri/Adri01/mp4_splitted\Frame_0676_14.645940000000149.jpg
../../data_export_1/Kinect/Adri/Adri01/mp4_splitted\Frame_0679_14.745960000000151.jpg
../../data_export_1/Kinect/Adri/Adri01/mp4_splitted\Frame_0739_16.74636000000016.jpg
../../data_export_1/Kinect/Adri/Adri01/mp4_splitted\Frame_0740_16.779700000000158.jpg
../../data_export_1/Kinect/Adri/Adri01/mp4_splitted\Frame_0741_16.813040000000157.jpg
../../data_export_1/Kinect/Adri/Adri01/mp4_splitted\Frame_0744_16.913060000000154.jpg
../../data_export_1/Kinect/Adri/Adri01/mp4_splitted\Frame_0745_16.946400000000153.jpg
../../data_export_1/Kinect/Adri/Adri01/mp4_splitted\Fra

### Checking output

In [20]:
csv_path = glob('../../data_export_1/Kinect/Xavi/Xavi01/mediapipe_processed_frames/*.csv')[0]
pd.read_csv(csv_path,header=None)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,13.29274,954.515419,318.791496,943.119278,336.701485,989.160919,330.210239,1005.044632,337.503895,1027.748222,309.353639,1043.793983,255.462561
1,13.32608,1040.297012,219.730999,937.392025,224.619003,1112.788582,227.925459,1029.566689,234.607136,1257.702141,219.222870,1153.254318,177.420208
2,13.35942,979.996605,213.828991,1037.719803,226.609642,1004.155083,265.286720,1142.148056,233.042389,1109.765854,254.238090,1257.856979,206.939002
3,13.39276,962.730904,221.598154,1050.159531,228.980307,943.874645,240.844688,1159.522591,240.237297,990.948601,194.593030,1260.718460,209.970263
4,13.42610,956.816540,208.841515,1060.099525,225.278279,940.964355,238.703561,1163.418159,236.935299,991.854744,200.974826,1278.828621,200.038311
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1679,69.27060,716.031990,224.068839,711.170940,214.135954,NaN,NaN,742.396145,303.221337,NaN,NaN,786.085281,326.536503
1680,69.30394,730.984383,219.384769,709.667358,215.112235,NaN,NaN,753.123150,318.713218,NaN,NaN,789.025040,383.364744
1681,69.33728,737.877789,221.190496,709.917870,220.314959,NaN,NaN,757.142944,325.663476,NaN,NaN,809.900837,399.127089
1682,69.37062,714.465466,221.892226,736.346798,222.489109,NaN,NaN,788.465195,328.807969,NaN,NaN,836.608200,402.982196
